In [ ]:
%run -i ../python/common.py
publish=False

if not publish:
    # cleanup any old state
    bashCmds('''[[ -d mydir ]] && rm -rf mydir
    [[ -a myfile ]] && rm myfile
    [[ -a errors ]] && rm errors 
    [[ -a mydate ]] && rm mydate
    [[ -a mynewdir ]] && rm -rf mynewdir
    [[ -a anotherfile ]] && rm anotherfile
    [[ -a mybin ]] && rm -rf  mybin
    [[ -a myinfo ]] && rm myinfo''')
else:
    bashCmds('''rm -rf ~/*''')
    
closeAllOpenTtySessions()
bash = BashSession()

generated="~/myfile ~/errors ~/mydate ~/mydir ~/mynewdir ~/out"


```{warning}
This portion of the book is under construction, not ready to be read
```

(cont:gs:abstractions)=
# Operating System Abstractions

UNIX has instilled in generations of computer scientists a basic aesthetic for how to design and structure complicated collections of software.  In particular one learns that the designers of UNIX tried to structure the system around a small core set of ideas, "abstractions", that once understood allows a programmer to understand the rest of the system and how to get things done.   In this chapter we briefly introduce some of those key abstractions, with the focus being: 1) to provide a conceptual overview, 2) provide enough information to get you started on writing a simple shell; the introductory assignment in many OS courses.  



## Files

A core idea of Unix is that everything is a file, where a file is a stream of bytes.  A process can use files to access durable state on a disk, communicate with other processes, find out information about the system, and communicate over the network.   This central idea enables one to write programs that do the same task irrespective of where the data is coming from or going to.  As we will see, it also allows programmers to tie together multiple simple programs to do a more complex task.  


### File Descriptors, read and write
As shown {numref}`file-desc` the kernel maintains a table of open files, and a file descriptor is an index into that table that can be used to read or write to a particular file.  Entries are added to the table by operations that open or create a file or create a special file like a network connection.  There are three special file descriptors shown in {numref}`fd-table`, that programs (and libraries) should use for input, output and errors. 

```{list-table} Standard well known file descriptors. 
:header-rows: 1
:name: fd-table
:width: 50%

* - Value
  - Name
  - Purpose
* - 0
  - stdin
  - standard input; process should read data from here
* - 1
  - stdout
  - standard output; process will write its output here
* - 2
  - stderr
  - standard error; process should write out errors here
```

The system calls that work on all files are:

- `n = write(desc, buffer, len)`: Write `len` bytes from `buffer` into a stream identified by `desc`.

- `n = read(desc, buffer, max)`: Read `max` bytes (or fewer if no data is available) from stream identified by `desc` into `buffer`and returning the actual number of bytes `n` read.  

To make files in the file system look like a stream, on each read or write operation, the kernel increases a per `desc` offset by the amount of data read or written.  Turns out that this naturally matches many applications that naturally read or write files in their entirety, and is key to enabling a common abstraction across all I/O. 


To understand how to use these operations, you really need to read the manual.  In Linux you can find out about everything using the **man** program.  For example:

In [ ]:
bash.run("man 2 write", height='1in')

```{note}
:class: margin
In this case, the *2* refers to the section of the manual for system calls.  To find out about the different sections, you, of course, read the manual about the man command.
```



### Composing programs together

So what's the big deal, well it turns out that if we can read and write streams, we can now write programs that combine together to do something very powerful.  For example, the **|** symbol tells the shell to take the output of one program and send it to the input of the next program.  So, lets say we are trying to find all the programs on our computer that have anything to do with perl, the following command will list the contents of the /usr/bin/ directory send the output of that listing to a grep program that searches for the word perl, and send the output of that to a program that counts the number of lines of input it had


In [ ]:
bash.run("ls /usr/bin/ | grep perl | wc -l", height='1in')

Linux now exposes all kinds of information through synthetic file systems, giving users and administrators massive ability to automate.  For example, in ```bash``` the shell we are using ```$$``` lets us know the ``id`` of the process we are in.  So, stealing a nice example from [jonathan](https://jappavoo.github.io/UndertheCovers/textbook/unix/shellintro.html#standard-output-and-redirection), the following command shows the 

In [ ]:
bash.run('''ls -l /proc/$$/fd/{0..2}''')

In [ ]:
bash_pid=bash.getPid()
bash_stdout=os.path.realpath("/proc/" + bash_pid.__str__() + "/fd/1")
bash.run("file " + bash_stdout)

And we can see that our stdin, stdout, and stderr all point to a character special file is in Unix is used to represent a terminal, and we can write to that same special device and it will appear in our terminal. 

In [ ]:
bash.run("echo \"hello class\" > " + bash_stdout)

The goal of this section was to discuss some of the power of the shell and the abstraction Unix has of everything being a file.  I would strongly encourage reading the shell and unix sections of [Under the Covers: The Secret Life of Software](https://jappavoo.github.io/UndertheCovers/textbook/intro_tb.html#under-the-covers-the-secret-life-of-software)  for a much more detailed coverage of this material.

### File system system calls

Read and write work for all files, there are also a number of system calls specific to file systems. While we discuss this in more detail [later](cont:fs:interface), we briefly introduce the key information you need to know.  First, it is important to realize that all Unix file systems organize information in a hierarchy as shown in numref}`fs:tree-logical-abs`.  

```{figure} ../images/pb-figures/fs/filesys-tree.png
---
width: 45%
name: fs:tree-logical-abs
---
Logical view: hierarchical file system name space
```

Some of the system calls specific to file systems are:

- `lseek(desc, offset, flag)` Set an open file's current position to that
specified by `offset` and `flag`, which specifies whether `offset` is
relative to the beginning, end, or current position in the file.

- `int desc = open(name, O_READ)`: Verify that file `name` exists and may
be read, and then return a *descriptor* which may be used to refer to
that file when reading it.

- `int desc = open(name, O_WRITE | flags, mode)`: Verify permissions and
open `name` for writing, creating it (or erasing existing contents) if
necessary as specified in `flags`. Returns a descriptor which may be
used for writing to that file.

- `close(desc)`: stop using this descriptor, and free any resources
allocated for it.

- `lseek(desc, offset, flag)`: Set an open file's current position to that
specified by `offset` and `flag`, which specifies whether `offset` is
relative to the beginning, end, or current position in the file.


### Pipe system calls


### Calls to manipulate the file descriptors


## saved for potential use

```{figure} ../images/intro/file-desc.png
---
width: 50%
name: file-desc
---
The kernel array of file descriptors.
```